# Lecture de la base de donnée

La base de donnée sera lue à partir d'un fichier csv correspondant en terme de structure à [example.csv](example.csv). <br>
Ce fichier contient quatres colonnes : fichier csv contenant le chromatograme, etat normal(1) ou non(0), label : description, le numéro de la catégorie entre 0 et 5.

Comme dans [readData.ipynb](readData.ipynb), on importe les modules utiles.

In [1]:
import pandas as pd
import tools
from importlib import reload
reload(tools)

pd.options.plotting.backend = "plotly"

On spécifie le chemin du fichier dans lequel est stocké la base de donnée dans la variable `FILE`, et le chemin jusqu'aux chromatogrammes mentionnés dans la base de donnée dans la variable : `PATH`. <br>

In [15]:
FILE = "../data/data_final/database.csv"
PATH = "../data/data_final/"

On met les informations de la base de donnée dans un DataFrame (`db`) qui permet d'avoir accès aux données de façon structurée.

In [24]:
db = pd.read_csv(FILE)
db.head()

,file,status,label,label_num
0,abdelhaks-20210226,1,Pas d'évidence de trouble d'ordre métabolique ...,0
1,abdullaha-20210426,0,Pas d'évidence de trouble d'ordre métabolique ...,3
2,achouria-20210209,0,Pas d'évidence de trouble d'ordre métabolique ...,4
3,achouria-20210322,0,Pas d'évidence de trouble d'ordre métabolique ...,4
4,aitbaddouk-20210519,1,Pas d'évidence de trouble d'ordre métabolique ...,0


On vérifie ensuite que le fichier de base de donnée n'a pas été compromis par un autre élément du projet.

In [25]:
# vérification qu'il n'y a pas d'erreur
temp = db[~db["status"].isin([0,1])]
print(temp.head())
temp = db[(db['status'] == 1) & (db['label_num'] != '0')]
print(temp)
temp = db[(db['status'] == 0) & (db['label_num'] == '0')]
print(temp)

Empty DataFrame
Columns: [file, status, label, label_num]
Index: []
Empty DataFrame
Columns: [file, status, label, label_num]
Index: []
Empty DataFrame
Columns: [file, status, label, label_num]
Index: []


Ensuite plutôt que d'avoir uniquement le nom des chromatogrammes, on peut récupérer les courbes traitées avec la fonction `readListOfData`.

In [134]:
#db['chromato'] = tools.readListOfData(db['file'], PATH)
#print(db)

In [135]:
#db['chromato'][0].df.plot()

Sélection des colonnes pour l'entrainement du réseau (dans le cas normal/non normal)

In [136]:
#db.loc[:, ['chromato', 'status']]

Pour connaitre l'efficacité d'un humain et donc avoir une base de comparaison pour l'algorithme d'intelligence artificielle, on peut ici donner un echantillon de données à diagnostiquer, toutes les catégories étant également représentés.

In [137]:
import random

print(len(db))
test_names = []
for i in range(6):
    temp = db[db['label_num'].str.contains(str(i)) == True]
    print(i, '  ', len(temp))
    for j in range(6):
        test_names.append(temp['file'].iloc[random.randrange(len(temp))])
random.shuffle(test_names)

for i in test_names:
    print(i)

566
0    299
1    25
2    65
3    76
4    91
5    27
bertrandp-20190201
settouchei-20190424
mangonim-20190717
syh-20190912
mastourab-20190219
chesinara-20191009
husica-20191108
bruyelleCn-20191017
dimolad-20190909
botafemmin-20190225
chesnaisv-20190715
colakeram-20190923
heriquem-20190329
durmc-20190724
piechockil-20190910
heriquem-20190329
genouxm-20190401
parisetn-20190116
wagnerg-20190620
durmc-20190802
rachidin-20191217
contall-20191203
usmana-20190520
rettele-20190509
contall-20191203
harutyunan-20190211
tassettii-20190412
bonjourg-20190713
calamel-20190925
picardl-20190618
bonomim-20191124
dosdas-20190924
damaouin-20191104
bruets-20190304
benyahiak-20190228
marechalj-20190518


Exemple de données sous forme de matrice pour l'utilisation en machine learning avec scikit-learn, et affichage du premier chromatogramme.
Chaque chromatogramme est représenté sur une ligne de la matrice.

In [138]:
"""import matplotlib.pyplot as plt
reload(tools)
X, y = tools.getData(FILE, PATH)
print('Nombre de données', len(y))
print("En entrée : \n", X, "\nEn sortie : ", y)
plt.plot(X[0, :])
plt.show()"""

'import matplotlib.pyplot as plt\nreload(tools)\nX, y = tools.getData(FILE, PATH)\nprint(\'Nombre de données\', len(y))\nprint("En entrée : \n", X, "\nEn sortie : ", y)\nplt.plot(X[0, :])\nplt.show()'

Dans le but de limiter le temps de calcul, dans la base de donnée finale utilisée, les fichiers enregistrés sont ceux après traitement, et les trois années sont rassemblées en un fichier.

In [1]:
import tools
import pandas as pd

files = ['database-2019', 'database-2020', 'database-2021']
paths = ['../data/2019', '../data/2020', '../data/2021']

database = open('../data/data_final/database.csv', 'w')
size = 0

for i in range(len(files)):
    # lecture de la base de donnée de l'année sélectionnée
    db = pd.read_csv('../data/' + files[i] + '.csv')
    chromatos = db['file'].to_numpy()
    # enregistrement de chaque chromatogramme après traitement
    for chromato in chromatos:
        try:
            data = tools.readAndAdaptDataFromCSV(paths[i], chromato)
        except tools.ReadDataException as e:
            print(chromato, '  ', e)
            continue
        data.df['values'].to_csv('../data/data_final/' + chromato + '.csv')
        size += 1
    # ajout des lignes corespondantes dans la base de donnée finale
    f = open('../data/' + files[i] + '.csv')
    content = f.read()
    database.write(content+'\n')
database.close()
print('taille de la base de donnée : ', size)
    


kaczorc-202011121    Le fichier chromatogramme n'a pas pu être trouvé
seghiri-20200813    Une des molécules est détectée en dehors de l'intervalle de temps de la mesure
taille de la base de donnée :  1370


Mise en place d'une base de donnée d'entrainement et de test

In [3]:
from shutil import copyfile

path = '../data/data_final/'
path_train = '../data/train/'
path_test = '../data/test/'


db = pd.read_csv(path + 'database.csv')
print('Taille originale : ', len(db))

train_set = db.sample(frac = 0.90)
test_set = db.drop(train_set.index)
print('Taille train_set : ', len(train_set), ' Taille test_set : ', len(test_set))

for file in train_set['file']:
    copyfile(path + file + '.csv', path_train + file + '.csv')

for file in test_set['file']:
    copyfile(path + file + '.csv', path_test + file + '.csv')

train_set.to_csv(path_train + 'database.csv')
test_set.to_csv(path_test + 'database.csv')

Taille originale :  1370
Taille train_set :  1233  Taille test_set :  137
